## Task 1: Create a cloud client

In [10]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /home/bstollnitz/git/config.json


## Task 2: Register the training data

In [11]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

DATA_NAME = 'hospital_train_parquet'

training_data = Data(
    name=DATA_NAME,
    path='data/training_data.parquet',
    type=AssetTypes.URI_FILE,
    description='RAI hospital train data'
)
tr_data = ml_client.data.create_or_update(training_data)

## Task 3: Create a compute cluster

In [12]:
from azure.ai.ml.entities import AmlCompute

COMPUTE_NAME = 'trainingcompute'

my_compute = AmlCompute(
    name=COMPUTE_NAME,
    size='Standard_DS2_v2',
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=3600
)
ml_client.compute.begin_create_or_update(my_compute)

## Task 4: Create the job

In [13]:
from azure.ai.ml import command, Input, Output

TARGET_COLUMN_NAME = 'readmit_status'

# Create the job.
job = command(
    description='Trains hospital readmission model',
    experiment_name='hospital_readmission',
    compute=COMPUTE_NAME,
    inputs=dict(training_data=Input(type='uri_file', path=f'{DATA_NAME}@latest'), 
                target_column_name=TARGET_COLUMN_NAME),
    outputs=dict(model_output=Output(type=AssetTypes.MLFLOW_MODEL)),
    code='src/',
    environment='azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4',
    command='python train.py ' + 
            '--training_data ${{inputs.training_data}} ' +
            '--target_column_name ${{inputs.target_column_name}} ' +
            '--model_output ${{outputs.model_output}}'
)
job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(job.name)

Uploading src (0.0 MBs): 100%|██████████| 2234/2234 [00:00<00:00, 150610.40it/s]




RunId: gentle_fennel_6myjpb0n20
Web View: https://ml.azure.com/runs/gentle_fennel_6myjpb0n20?wsid=/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/workspaces/mlws-bstollnitz

Execution Summary
RunId: gentle_fennel_6myjpb0n20
Web View: https://ml.azure.com/runs/gentle_fennel_6myjpb0n20?wsid=/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/workspaces/mlws-bstollnitz



## Task 5: Register the model

In [16]:
from azure.ai.ml.entities import Model

MODEL_NAME = 'hospital_readmission_model'

# Register the model.
model_path = f"azureml://jobs/{job.name}/outputs/model_output"
model = Model(name=MODEL_NAME,
                path=model_path,
                type=AssetTypes.MLFLOW_MODEL)
registered_model = ml_client.models.create_or_update(model)

## Task 6: Deploy the model

In [17]:
from azure.ai.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint 

ENDPOINT_NAME = 'hospital-readmission-endpoint'
DEPLOYMENT_NAME = 'blue'

# Create the managed online endpoint.
endpoint = ManagedOnlineEndpoint(
    name=ENDPOINT_NAME,
    auth_mode='key',
)
registered_endpoint = ml_client.online_endpoints.begin_create_or_update(
    endpoint)

# Create the managed online deployment.
deployment = ManagedOnlineDeployment(name=DEPLOYMENT_NAME,
                                        endpoint_name=ENDPOINT_NAME,
                                        model=registered_model,
                                        instance_type='Standard_DS4_v2',
                                        instance_count=1)
ml_client.online_deployments.begin_create_or_update(deployment)

# Set deployment traffic to 100%.
registered_endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.online_endpoints.begin_create_or_update(
    registered_endpoint)

Check: endpoint hospital-readmission-endpoint exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored
Creating/updating online deployment blue 

......................................................................................................

Done (8m 58s)


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://hospital-readmission-endpoint.westus2.inference.ml.azure.com/score', 'swagger_uri': 'https://hospital-readmission-endpoint.westus2.inference.ml.azure.com/swagger.json', 'name': 'hospital-readmission-endpoint', 'description': None, 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/providers/microsoft.machinelearningservices/workspaces/mlws-bstollnitz/onlineendpoints/hospital-readmission-endpoint', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/providers/Microsoft.MachineLearningServices/locations/westus2/mfeOperationsStatus/oe:4b0cadf3-7f66-44b6-ad51-ad8d6f93d8d5:ea31256c-5c3d-44c1-83b6-2e48f77596ce?api-version=2022-02-01-preview'}, 'id': '/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourceGroups/rg-bstollnitz/provide

## Task 7: Invoke the endpoint

In [18]:
TEST_DATA_PATH="test_data.json"

# Invoke the endpoint.
result = ml_client.online_endpoints.invoke(endpoint_name=ENDPOINT_NAME, request_file=TEST_DATA_PATH)
print(result)

["not readmitted"]
